In [1]:
import numpy as np

In [3]:
softmax_outputs=np.array([[0.7,0.1,0.2],[0.1,0.5,0.4],[0.02,0.9,0.08]])
class_targets=[0,1,1]
print(softmax_outputs[[0,1,2],class_targets])

[0.7 0.5 0.9]


In [6]:
# Applying cross entropy loss
print(-np.log(softmax_outputs[range(len(softmax_outputs)),class_targets]))
neg_log=-np.log(softmax_outputs[range(len(softmax_outputs)),class_targets])
average_loss=np.mean(neg_log)
print(average_loss)

[0.35667494 0.69314718 0.10536052]
0.38506088005216804


In [14]:
# If data is one hot encoded
y_true_check=np.array([[1,0,0],[0,1,0],[0,1,0]])
y_pred_clipped_check=np.array([[0.7,0.2,0.1],[0.1,0.5,0.4],[0.02,0.9,0.08]])

A=y_true_check*y_pred_clipped_check
B=np.sum(A,axis=1)
C=-np.log(B)
print(C)
print(np.mean(C))

[0.35667494 0.69314718 0.10536052]
0.38506088005216804


In [16]:
## Common Loss class
class Loss:
    def calculate(self, output,y):
        sample_losses=self.forward(output,y)
        # calculate mean loss
        data_loss=np.mean(sample_losses)
        return data_loss

In [17]:
# Implementing Loss class
class Loss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred,y_true):
        # number of samples in a batch
        samples=len(y_pred)
        y_pred_clipped=np.clip(y_pred,1e-7,1-1e-7)
        # Categorical (1D)
        if len(y_true.shape)==1:
            correct_confidence=y_pred_clipped[range(samples),y_true]

        # for one hot encoded (2D)
        elif len(y_true.shape)==2:
            correct_confidence=np.sum(y_pred_clipped*y_true,axis=1)

        negative_log_likelihood=-np.log(correct_confidence)
        return negative_log_likelihood

         

In [19]:
softmax_outputs=np.array([[0.7,0.1,0.2],[0.1,0.5,0.4],[0.02,0.9,0.08]])
class_targets=np.array([[1,0,0],[0,1,0],[0,1,0]])
loss_function=Loss_CategoricalCrossEntropy()
loss=loss_function.calculate(softmax_outputs,class_targets)
print(loss)

0.38506088005216804


In [21]:
## Loading the previous codes
class Layer_Dense:
    # Layer initialization
    def __init__(self,n_inputs,n_neurons):
        # Initialize weights and biases
        self.weights=0.01*np.random.randn(n_inputs,n_neurons)
        self.biases=np.zeros((1,n_neurons))
        

    ## Forward pass
    def forward(self, inputs):
        ### Calculate output values from inputs, weights and biases
        self.output=np.dot(inputs,self.weights)+self.biases


### ACTIVATION FUNCTION: RELU
class Activation_ReLU:
    # Forward pass
    def forward(self,inputs):
        # Calculate output values from input
        self.output=np.maximum(0,inputs)



#Softmax activation
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # get unnormalised probabilities
        exp_values=np.exp(inputs-np.max(inputs,axis=1,keepdims=True))
        # Normalise them for each sample
        probablitiles=exp_values/np.sum(exp_values,axis=1,keepdims=True)
        self.output=probablitiles

In [23]:
### NEURAL NETWORK FORWARD PASS (with LOSS)
from nnfs.datasets import spiral_data
# Create dataset
X,y=spiral_data(samples=100,classes=3)
# create dense layer with 2 input features and 3 output values
dense1=Layer_Dense(2,3)
# create Relu activation
activation1=Activation_ReLU()

#Create second dense layer with 3 input features
dense2=Layer_Dense(3,3) # 3-inputs and 3-neurons
activation2=Activation_Softmax()

## create loss function
loss_function=Loss_CategoricalCrossEntropy()

# make a forward pass of our training data through this layer
dense1.forward(X) # generate outputs from 1st layer of neuron

## Make  a forward pass through activation function
#it takes output of first dense layer here
activation1.forward(dense1.output)
# forward pass through second dense layer
# it takes output of activation function of first layer as inputs
dense2.forward(activation1.output) # use output of previous activation as input
## Make  a forward pass through activation function
#it takes output of second dense layer here
activation2.forward(dense2.output)
# lets see output of first few samples:
print(activation2.output[:5])

loss=loss_function.calculate(activation2.output,y)
print('loss:',loss)


### calculate accuracy from outputs of activation2 and targets
predictions=np.argmax(activation2.output,axis=1)
if len(y.shape)==2:
    y=np.argmax(y,axis=1)

accurcay=np.mean(predictions==y)
print('acc:',accurcay)

[[0.33333333 0.33333333 0.33333333]
 [0.33333323 0.33333333 0.33333343]
 [0.33333297 0.33333358 0.33333345]
 [0.33333297 0.33333341 0.33333363]
 [0.33333275 0.33333355 0.33333369]]
loss: 1.098610444420201
acc: 0.37333333333333335


In [25]:
### Inroducing Accurcay
softmax_outputs=np.array([[0.7,0.2,0.1],[0.1,0.5,0.4],[0.02,0.9,0.08]])
class_targets=np.array([[0,1,1]])

predictions=np.argmax(softmax_outputs,axis=1)
if len(class_targets.shape)==2:
    class_targets=np.argmax(class_targets,axis=1)

accurcay=np.mean(predictions==class_targets)
print('acc:',accurcay)

acc: 0.6666666666666666
